# Install*

In [1]:
!pip install mxnet-cu101
!pip install gluonnlp
!pip install sentencepiece
!pip install transformers==2.2.2
!pip install urllib3==1.25.10
# 실행 후 런타임 다시시작 실행 (urllib3를 다시 로드 해야 함)

     |████████████████████████████████| 358.1 MB 4.6 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 10.2 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595720 sha256=4a71ab6dc27611bf00fe6dfe5612ee10eedb12e0f9c84a41a61d69154b842570
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 8.2 MB/s 
     |████████████████████████████████| 387 kB 8.1 MB/s 
     |████████████████████████████████| 132 kB 66.2 MB/s 
     |████████████████████████████████| 895 kB 61.4 MB/s 
     |████████████████████████████████| 79 kB 7.8 MB/s 
     |████████████████████████████████| 8.6 MB 56.6 MB/s 
     |████████████████████████████████| 138 kB 57.4 MB/s 
     |██████████

In [2]:
# kobert install
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-qxr1_f_d
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-qxr1_f_d
     |████████████████████████████████| 47.3 MB 156 kB/s 
     |████████████████████████████████| 4.5 MB 36.9 MB/s 
     |████████████████████████████████| 3.8 MB 52.6 MB/s 
     |████████████████████████████████| 6.5 MB 46.1 MB/s 
     |████████████████████████████████| 596 kB 54.9 MB/s 
     |████████████████████████████████| 67 kB 5.9 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15674 sha256=d3016f8e14985ebca431871d2defbb0665e13332971dc667eb55c5c3a1568959
  Stored in directory: /tmp/pip-ephem-wheel-cache-u6m0uge0/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYA

# Evn*

In [3]:
# imports
import argparse
import os
import random
import shutil
import json
import zipfile
import math
import copy
import collections
import re

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import sentencepiece as spm
import gluonnlp as nlp
import torch
import transformers
import kobert
from kobert.pytorch_kobert import get_pytorch_kobert_model

from tqdm.notebook import tqdm, trange

In [4]:
# 환경 설정
args = {
    # random seed value
    "seed": 1234
}
args = argparse.Namespace(**args)

print(args)

Namespace(seed=1234)


In [5]:
# random seed 설정
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(args.seed)

In [6]:
# gpu 사용량 확인
!nvidia-smi

Mon Mar 21 11:09:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# KoBERT*

In [7]:
# cpu or gpu
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# model weights download
bert_model, bert_vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


# Data*

In [8]:
!wget https://github.com/e9t/nsmc/raw/master/ratings_train.txt
!wget https://github.com/e9t/nsmc/raw/master/ratings_test.txt

--2022-03-21 11:09:45--  https://github.com/e9t/nsmc/raw/master/ratings_train.txt
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt [following]
--2022-03-21 11:09:45--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: ‘ratings_train.txt’

ratings_train.txt   100%[===================>]  13.95M  --.-KB/s    in 0.08s   

2022-03-21 11:09:46 (173 MB/s) - ‘ratings_train.txt’ saved [14628807/14628807]

--2022-03-21 11:09:46--  https://github.com/e9t/nsmc/raw/mas

In [9]:
id_to_label = {0: "부정", 1: "긍정"}

# Train, Test 데이터 생성*

In [10]:
tokenizer = kobert.utils.get_tokenizer()
bert_tokenizer = nlp.data.BERTSPTokenizer(tokenizer, bert_vocab, lower=False)
bert_tokenizer

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [11]:
df_train = pd.read_csv("ratings_train.txt", header=0, delimiter='\t')
df_train = df_train.dropna()
df_train

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [12]:
df_test = pd.read_csv("ratings_test.txt", header=0, delimiter='\t')
df_test = df_test.dropna()
df_test

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


In [13]:
class NSMCDataset(torch.utils.data.Dataset):
    def __init__(self, df, bert_tokenizer, n_seq, pad=True, pair=False):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=n_seq, pad=pad, pair=pair)
        self.sentences = []
        self.labels = []
        for i, row in df.iterrows():
            if type(row["document"]) == str:
                self.sentences.append(transform([row["document"]]))
                self.labels.append(np.int32(row["label"]))

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [14]:
n_seq = 64
data_train = NSMCDataset(df_train, bert_tokenizer, n_seq)
data_test = NSMCDataset(df_test, bert_tokenizer, n_seq)

In [15]:
len(data_train), data_train.__getitem__(2)

(149995,
 (array([   2, 1458, 7191,    0, 6888, 5540, 6553, 6369, 5398, 7088, 7461,
         7422, 7831,    3,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
  array(14, dtype=int32),
  array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        dtype=int32),
  0))

In [16]:
def collate_fn(batch):
    token_ids, valid_length, segment_ids, labels = list(zip(*batch))

    return [
        torch.tensor(token_ids),
        torch.tensor(np.stack(valid_length)),
        torch.tensor(segment_ids),
        torch.tensor(labels),
    ]

In [17]:
batch_size = 64
sampler = torch.utils.data.sampler.RandomSampler(data_train)
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, sampler=sampler)
# train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, collate_fn=collate_fn)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size)
# test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, collate_fn=collate_fn)

# Train

In [18]:
class NSMCClassifier(torch.nn.Module):
    def __init__(self, bert_model, d_model=768, n_output=2, dr_rate=0.1):
        super().__init__()
        self.bert_model = bert_model
        self.dr_rate = dr_rate
                 
        self.classifier = torch.nn.Linear(d_model , n_output)
        self.dropout = torch.nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert_model(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        out = self.dropout(pooler)
        return self.classifier(out)

In [19]:
model = NSMCClassifier(bert_model).to(device)
model

NSMCClassifier(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affin

In [20]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [21]:
optimizer = transformers.AdamW(optimizer_grouped_parameters, lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [22]:
num_epochs = 1
warmup_ratio = 0.1
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

t_total, warmup_step

(2344, 234)

In [23]:
scheduler = transformers.optimization.get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [24]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy() / max_indices.size()[0]
    return train_acc

In [25]:
max_grad_norm = 1

best_acc = 0.0
n_step = 0

p_bar = trange(t_total)
for e in range(num_epochs):
    train_loss, train_acc = [], []
    test_loss, test_acc = [], []

    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, labels) in enumerate(train_dataloader):
        optimizer.zero_grad()

        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        labels = labels.long().to(device)

        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule

        train_loss.append(loss.item())
        train_acc.append(calc_accuracy(out, labels).item())

        p_bar.update(1)
        n_step += 1
        if n_step % 10 == 0:
            p_bar.set_description(f'epoch {e+1} loss {np.mean(train_loss):0.4f} acc {np.mean(train_acc):0.4f}')
    p_bar.set_description(f'epoch {e+1} loss {np.mean(train_loss):0.4f} acc {np.mean(train_acc):0.4f}')

    with torch.no_grad():
        model.eval()
        for batch_id, (token_ids, valid_length, segment_ids, labels) in enumerate(test_dataloader):
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            valid_length = valid_length
            labels = labels.long().to(device)

            out = model(token_ids, valid_length, segment_ids)
            loss = loss_fn(out, labels)

            test_loss.append(loss.item())
            test_acc.append(calc_accuracy(out, labels).item())
    
    val_acc = np.mean(test_acc)
    if best_acc < val_acc:
        torch.save(model.state_dict(), 'kobert_nsmc.pth')
        print(f'prev {best_acc:0.4f}, curr {val_acc:0.4f} improved and save')
        best_acc = val_acc
    else:
        print(f'curr {val_acc:0.4f}, best {best_acc:0.4f} not improved')

  0%|          | 0/2344 [00:00<?, ?it/s]

prev 0.0000, curr 0.8924 improved and save


## 평가

In [26]:
model = NSMCClassifier(bert_model).to(device)
model.load_state_dict(torch.load('kobert_nsmc.pth'))

<All keys matched successfully>

In [27]:
loss_fn = torch.nn.CrossEntropyLoss()

In [28]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy() / max_indices.size()[0]
    return train_acc

In [29]:
test_loss, test_acc = [], []
y_true, y_pred = [], []

with torch.no_grad():
    model.eval()
    for token_ids, valid_length, segment_ids, labels in tqdm(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        labels = labels.long().to(device)

        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, labels)

        test_loss.append(loss.item())
        test_acc.append(calc_accuracy(out, labels).item())

        y_true.append(labels.cpu().numpy())
        y_pred.append(out.cpu().numpy())

np.mean(test_loss), np.mean(test_acc)

  0%|          | 0/782 [00:00<?, ?it/s]

(0.2579795128339544, 0.8924447914617352)

In [30]:
y_true = np.concatenate(y_true)
y_true

array([1, 0, 0, ..., 0, 0, 0])

In [31]:
y_pred = np.concatenate(y_pred)
y_pred

array([[-2.1618028 ,  2.1682813 ],
       [-0.479235  ,  0.54110765],
       [ 0.05327683, -0.03861513],
       ...,
       [-0.5965573 ,  0.5942447 ],
       [ 2.978435  , -2.5437276 ],
       [ 1.142779  , -0.9990089 ]], dtype=float32)

In [32]:
y_class = np.argmax(y_pred, axis=-1)
y_class

array([1, 1, 0, ..., 1, 0, 0])

In [33]:
cf_matrix = np.zeros((2, 2)).astype(np.int)
cf_matrix

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.


array([[0, 0],
       [0, 0]])

In [34]:
for t, p in zip(y_true, y_class):
    cf_matrix[int(t), int(p)] += 1
cf_matrix

array([[22169,  2657],
       [ 2722, 22449]])

In [35]:
tp = cf_matrix[1, 1]
tn = cf_matrix[0, 0]
fp = cf_matrix[0, 1]
fn = cf_matrix[1, 0]

accuracy = (tp + tn) / max((tp + tn + fp + fn), 1)
print(f'accuracy: {accuracy}')
precision = (tp) / max((tp + fp), 1)
print(f'precision: {precision}')
recall = (tp) / max((tp + fn), 1)
print(f'recall: {recall}')
f1 = 2 * (precision * recall) / max((precision + recall), 1)
print(f'f1: {f1}')

accuracy: 0.8924135448126888
precision: 0.8941687246076635
recall: 0.8918596797902348
f1: 0.8930127095888777


## 배포

In [36]:
model = NSMCClassifier(bert_model).to(device)
model.load_state_dict(torch.load('kobert_nsmc.pth'))

<All keys matched successfully>

In [37]:
string = "영화 너무 심심해"

In [38]:
transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=60, pad=False, pair=False)

In [39]:
token_ids, valid_length, segment_ids = transform([string])
token_ids, valid_length, segment_ids

(array([   2, 3394, 1458, 3060, 6745, 7848,    3], dtype=int32),
 array(7, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0], dtype=int32))

In [40]:
token_ids = torch.tensor([token_ids]).to(device)
valid_length = torch.tensor(np.array([valid_length])).to(device)
segment_ids = torch.tensor([segment_ids]).to(device)

token_ids.shape, valid_length.shape, segment_ids.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  """Entry point for launching an IPython kernel.


(torch.Size([1, 7]), torch.Size([1]), torch.Size([1, 7]))

In [41]:
with torch.no_grad():
    model.eval()
    y_pred = model(token_ids, valid_length, segment_ids).cpu().numpy()
y_pred

array([[ 2.8272648, -2.357929 ]], dtype=float32)

In [42]:
y_class = np.argmax(y_pred, axis=-1)[0]
y_class

0

In [43]:
print(f"{string} : {id_to_label[y_class]}")

영화 너무 심심해 : 부정


In [44]:
def do_predict(transform, model, string):
    token_ids, valid_length, segment_ids = transform([string])

    token_ids = torch.tensor([token_ids]).to(device)
    valid_length = torch.tensor(np.array([valid_length])).to(device)
    segment_ids = torch.tensor([segment_ids]).to(device)

    with torch.no_grad():
        model.eval()
        y_pred = model(token_ids, valid_length, segment_ids).cpu().numpy()
    y_class = np.argmax(y_pred, axis=-1)[0]
    return id_to_label[y_class]

In [45]:
do_predict(transform, model, string)

'부정'

In [46]:
while True:
    print("input> ", end="")
    string = str(input())
    if len(string) == 0:
        break
    result = do_predict(transform, model, string)
    print(result)

input> 
